In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
  torch==1.13.1 \
  torchdata==0.5.1 --quiet

%pip install \
  transformers==4.27.2 \
  datasets==2.11.0 \
  evaluate==0.4.0 \
  peft==0.3.0 \
  loralib==0.1.1 \
  rouge_score==0.1.2 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.7 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-multimodal 0.8.3 requires pytorch-lightning<1.10.0,>=1.9.0, but you have pytorch-lightning 2.0.9 which is incompatible.
autogluon-multimodal 0.8.3 requires scikit-learn<1.4.1,>=1.1, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 0.8.3 requires torchmetrics<0.12.0,>=0.11.0, but you have torchmetrics 1.0.3 which is incompatible.
autogluon-multimodal 0.8.3 requires torchvision<0.15.0, but y

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import evaluate 
import time
import peft
import loralib
import pandas as pd
import numpy as np
import rouge_score

In [4]:
dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(dataset_name)
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/sagemaker-user/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [6]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# 

In [7]:
index = 200
dashline = "-" * 100


dialogue = dataset['train'][index]['dialogue']
summary = dataset['train'][index]['summary']

inputs = tokenizer(dialogue, return_tensors="pt")
outputs = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens = 100)[0], skip_special_tokens = True)


print(dashline)
print("EXAMPLE ")
print(dashline)
print("DIALOGUE")
print(dialogue)
print(dashline)
print("SUMMARY BY HUMAN")
print(summary)
print(dashline)
print("SUMMARY BY MODEL -> WITH ZERO SHOT")
print(outputs)
print(dashline)

----------------------------------------------------------------------------------------------------
EXAMPLE 
----------------------------------------------------------------------------------------------------
DIALOGUE
#Person1#: What do you want to know about me?
#Person2#: How about your academic records at college?
#Person1#: The average grade of all my courses is above 85.
#Person2#: In which subject did you get the highest marks?
#Person1#: In mathematics I got a 98.
#Person2#: Have you received any scholarships?
#Person1#: Yes, I have, and three times in total.
#Person2#: Have you been a class leader?
#Person1#: I have been a class commissary in charge of studies for two years.
#Person2#: Did you join in any club activities?
#Person1#: I was an aerobics team member in college.
#Person2#: What sport are you good at?
#Person1#: I am good at sprint and table tennis.
#Person2#: You are excellent.
---------------------------------------------------------------------------------------

In [8]:
def tokenize_func(example):
  start_of_prompt = "Summarize the following dialogue:\n"
  end_of_prompt = "\nHere is the summarized version of the prompt:\n"
  prompt = [start_of_prompt + diag + end_of_prompt for diag in example["dialogue"]]
  example['input_ids'] = tokenizer(prompt, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt").input_ids  
  example['labels'] = tokenizer(example["summary"], max_length = 128, truncation = True, padding = "max_length", return_tensors = "pt").input_ids
  return example


tokenize_dataset = dataset.map(tokenize_func, batched = True, remove_columns = ['dialogue', 'summary', 'id', 'topic'])
tokenize_dataset['train'][0]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'input_ids': [12198,
  1635,
  1737,
  8,
  826,
  7478,
  10,
  1713,
  345,
  13515,
  536,
  4663,
  10,
  2018,
  6,
  1363,
  5,
  3931,
  5,
  27,
  31,
  51,
  7582,
  12833,
  77,
  7,
  5,
  1615,
  33,
  25,
  270,
  469,
  58,
  1713,
  345,
  13515,
  357,
  4663,
  10,
  27,
  435,
  34,
  133,
  36,
  3,
  9,
  207,
  800,
  12,
  129,
  3,
  9,
  691,
  18,
  413,
  5,
  1713,
  345,
  13515,
  536,
  4663,
  10,
  2163,
  6,
  168,
  6,
  25,
  43,
  29,
  31,
  17,
  141,
  80,
  21,
  305,
  203,
  5,
  148,
  225,
  43,
  80,
  334,
  215,
  5,
  1713,
  345,
  13515,
  357,
  4663,
  10,
  27,
  214,
  5,
  27,
  2320,
  38,
  307,
  38,
  132,
  19,
  1327,
  1786,
  6,
  572,
  281,
  217,
  8,
  2472,
  58,
  1713,
  345,
  13515,
  536,
  4663,
  10,
  1548,
  6,
  8,
  200,
  194,
  12,
  1792,
  2261,
  21154,
  19,
  12,
  253,
  91,
  81,
  135,
  778,
  5,
  264,
  653,
  12,
  369,
  44,
  709,
  728,
  3,
  9,
  215,
  21,
  39,
  293,
  207,
  5,
  1713

In [9]:
tokenized_datasets = tokenize_dataset.filter(lambda example, index: index % 100 == 0, with_indices = True)
tokenized_datasets

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})

In [13]:
output_dir = f'./dialogue_summ-{str(int(time.time()))}'

training_arguments = TrainingArguments(
    output_dir = output_dir,
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    learning_rate = 1e-5,
    weight_decay = 0.01,
    # per_device_train_batch_size = 1,
    # per_device_eval_batch_size = 1,
    # gradient_accumulation_steps = 1,
    num_train_epochs = 5,
    # save_total_limit = 3
    logging_steps = 1,
    # save_steps = 1
    # warmup_steps = 500
    # fp16 = False,
    # fp16_backend = "bf16"
    # fp16_full_eval = True
    max_steps = 3
)

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"]
)

# torch.cuda.empty_cache()

In [14]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,31.000000
2,36.500000
3,33.000000


TrainOutput(global_step=3, training_loss=33.5, metrics={'train_runtime': 241.215, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.012, 'total_flos': 32868352917504.0, 'train_loss': 33.5, 'epoch': 0.19})

In [15]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/generation_config.json to flan-dialogue-summary-checkpoint/generation_config.json
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/scheduler.pt to flan-dialogue-summary-checkpoint/scheduler.pt
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/trainer_state.json to flan-dialogue-summary-checkpoint/trainer_state.json
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/rng_state.pth to flan-dialogue-summary-checkpoint/rng_state.pth
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/config.json to flan-dial

In [16]:
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-r--r-- 1 sagemaker-user users 945M May 15  2023 ./flan-dialogue-summary-checkpoint/pytorch_model.bin


In [17]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype = torch.bfloat16)